<img src="https://github.com/RadimKozl/OpenCV_Project4/blob/832738cf629a0d6518297f860fd1952e3eaa3143/notebooks/img/OpenCV_logo.png" alt="OpenCV logo" style="width: 200px;"/>

###### <font size="10">YOLO models, base properties</font>

# Introduction

In this notebook, we will describe the basic properties of trained onnx models.

# Housekeeping

In [1]:
# !pip install opencv-python
# !pip install opencv-contrib-python
# !pip install opencv-python-headless
# !pip install opencv-contrib-python-headless
# !pip install ultralytics
# !pip install pillow
# !pip install onnx
# !pip install onnxruntime 

In [2]:
import os
os.chdir('C:\\Users\\kozl\\ITProjects\\DataScienciPython\\data_science_python_training\\lessons\\OpenCV\\lesson26\\coursework\\notebooks')

In [3]:
import cv2, sys, os, csv

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path

import pickle

from dataPath import MODEL_PATH, IMAGES_PATH, RESULTS_PATH, PICKLE_PATH

from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import onnxruntime as ort
from IPython.display import display

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, Output
from ipywidgets.widgets.interaction import interactive
from IPython.display import display

%matplotlib inline

In [4]:
matplotlib.rcParams['figure.figsize'] = (8.0,8.0)
matplotlib.rcParams['image.cmap'] = 'gray'
matplotlib.rcParams['image.interpolation'] = 'bilinear'

### ***Defining the necessary paths***

In [5]:
# path to the model YOLO
path_model_dir = os.path.join(MODEL_PATH, 'onnx')
# path to the *.pt files of models
path_pt_dir = os.path.join(MODEL_PATH, 'pt')
# path directory for load test images
path_images_dir = os.path.join(IMAGES_PATH, 'test_images')

### ***Loading ONNX models***

In [6]:
def load_onnx(path_dir):
    list_files = os.listdir(path_dir)
    list_onnx_files = [file for file in list_files if os.path.isfile(os.path.join(path_dir, file)) and file.endswith('.onnx')]
    file_paths = []
    for file in list_onnx_files:
        path = os.path.join(path_dir, file)
        file_paths.append(path)
    
    return file_paths

In [7]:
list_onnx_files = load_onnx(path_model_dir)
list_onnx_files

['..\\data\\models\\onnx\\YOLOv10m_FMD_01.onnx',
 '..\\data\\models\\onnx\\YOLOv10m_FMD_02.onnx',
 '..\\data\\models\\onnx\\YOLOv5m_FMD.onnx',
 '..\\data\\models\\onnx\\YOLOv6m_FMD_01.onnx',
 '..\\data\\models\\onnx\\YOLOv6m_FMD_02.onnx',
 '..\\data\\models\\onnx\\YOLOv7_FMD_01.onnx',
 '..\\data\\models\\onnx\\YOLOv7_FMD_02.onnx',
 '..\\data\\models\\onnx\\YOLOv7_FMD_03.onnx',
 '..\\data\\models\\onnx\\YOLOv8m_FMD.onnx',
 '..\\data\\models\\onnx\\YOLOv9m_FMD.onnx']

In [8]:
model_properties = {}

In [9]:
# Python code to merge dict 
def Merge(dict1, dict2, dict3):
    res_sub = {**dict1, **dict2}
    res = {**res_sub, **dict3}
    return res

In [10]:
for file in list_onnx_files:
    name_model_file = os.path.split(file)[1]
    name = name_model_file.split('.')[0]
    suffix = name_model_file.split('.')[1]
    bite_size = Path(file).stat().st_size

    properties_dict = {
        'Suffix': suffix, 
        'Bite_size (KB)': float(bite_size/1024)
    }
    
    # Load the exported ONNX model
    model = ort.InferenceSession(file, providers=['CPUExecutionProvider'])

    # Read inputs of model
    inputs = model.get_inputs();
    input = inputs[0]
    inputs_dict = {
        'Inputs_number': len(inputs),
        'Input_name': input.name,
        'Input_type': input.type,
        'Input_shape': input.shape
    }

    # Read outputs of model
    outputs = model.get_outputs()
    output = outputs[0]
    output_dict = {
        'Output_number': len(outputs),
        'Output_name': output.name,
        'Output_type': output.type,
        'Output_shape': output.shape
    }

    all_data = Merge(properties_dict, inputs_dict, output_dict)
    model_properties[name] = all_data

In [11]:
#model_properties

In [12]:
df_results = pd.DataFrame(model_properties).T
df_results 

,Suffix,Bite_size (KB),Inputs_number,Input_name,Input_type,Input_shape,Output_number,Output_name,Output_type,Output_shape
YOLOv10m_FMD_01,onnx,60078.87793,1,images,tensor(float),"[1, 3, 640, 640]",1,output0,tensor(float),"[1, 300, 6]"
YOLOv10m_FMD_02,onnx,67443.135742,1,input,tensor(float),"[1, 3, 640, 640]",7,output,tensor(float),"[1, 300, 6]"
YOLOv5m_FMD,onnx,98067.397461,1,images,tensor(float),"[1, 3, 640, 640]",1,output0,tensor(float),"[1, 6, 8400]"
YOLOv6m_FMD_01,onnx,310845.573242,1,images,tensor(float),"[1, 3, 640, 640]",1,outputs,tensor(float),"[1, 8500, 7]"
YOLOv6m_FMD_02,onnx,337215.750977,1,input,tensor(float),"[1, 3, 640, 640]",1,output,tensor(float),"[1, 8500, 7]"
YOLOv7_FMD_01,onnx,142643.126953,1,images,tensor(float),"[1, 3, 640, 640]",4,output0,tensor(float),"[1, 25200, 7]"
YOLOv7_FMD_02,onnx,142572.557617,1,images,tensor(float),"[1, 3, 640, 640]",3,output,tensor(float),"[1, 3, 80, 80, 7]"
YOLOv7_FMD_03,onnx,148011.740234,1,input,tensor(float),"[1, 3, 640, 640]",4,output,tensor(float),"[1, 25200, 7]"
YOLOv8m_FMD,onnx,101163.772461,1,images,tensor(float),"[1, 3, 640, 640]",1,output0,tensor(float),"[1, 6, 8400]"
YOLOv9m_FMD,onnx,78461.026367,1,images,tensor(float),"[1, 3, 640, 640]",1,output0,tensor(float),"[1, 6, 8400]"


### ***Loading PyTorch models***

In [13]:
def load_pt(path_dir):
    list_files = os.listdir(path_dir)
    list_pt_files = [file for file in list_files if os.path.isfile(os.path.join(path_dir, file)) and file.endswith('.pt')]
    file_paths = []
    for file in list_pt_files:
        path = os.path.join(path_dir, file)
        file_paths.append(path)
    
    return file_paths

In [14]:
list_pt_files = load_pt(path_pt_dir)
list_pt_files

['..\\data\\models\\pt\\YOLOv10m_FMD.pt',
 '..\\data\\models\\pt\\YOLOv5m_FMD.pt',
 '..\\data\\models\\pt\\YOLOv6m_FMD.pt',
 '..\\data\\models\\pt\\YOLOv7_FMD.pt',
 '..\\data\\models\\pt\\YOLOv8m_FMD.pt',
 '..\\data\\models\\pt\\YOLOv9m_FMD.pt']

In [15]:
pt_model_properties = {}

In [16]:
for file in list_pt_files:
    name_model_file = os.path.split(file)[1]
    name = name_model_file.split('.')[0]
    suffix = name_model_file.split('.')[1]
    bite_size = Path(file).stat().st_size

    properties_dict = {
        'Suffix': suffix, 
        'Bite_size (KB)': float(bite_size/1024)
    }
    
    pt_model_properties[name] = properties_dict

In [17]:
 #pt_model_properties

In [18]:
df_pt_results = pd.DataFrame(pt_model_properties).T
df_pt_results

,Suffix,Bite_size (KB)
YOLOv10m_FMD,pt,32680.991211
YOLOv5m_FMD,pt,49304.100586
YOLOv6m_FMD,pt,167715.404297
YOLOv7_FMD,pt,73044.056641
YOLOv8m_FMD,pt,50796.760742
YOLOv9m_FMD,pt,39861.838867
